입력 설정:

input_dir: CSV 파일이 저장된 디렉토리 경로.
output_csv: 앙상블 결과를 저장할 파일 경로.
iou_threshold: IOU 기준값(박스/마스크 병합 기준).
vote_threshold: 특정 예측을 유지하기 위한 최소 모델 투표 수.
RLE 처리 함수:

decode_rle: RLE 데이터를 binary mask로 변환.
encode_rle: binary mask를 RLE 데이터로 변환.
앙상블 수행:

동일한 image_name과 class에 대한 RLE를 IOU와 다수결 기준으로 병합.
IOU가 기준치를 넘는 마스크만 병합.
다수결 기준(vote_threshold)을 만족하는 마스크만 최종 결과에 포함.
결과 저장:

결과를 output_csv로 저장.

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [2]:
# 입력 설정

# CSV 파일이 위치한 디렉토리 경로
input_dir = "/data/ephemeral/home/kjh2/level2-cv-semanticsegmentation-cv-13-lv3/inference/"

# 결과를 저장할 파일명
output_csv = "csv_ensemble_soft_1.csv"  

In [3]:
# GPU/CPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# RLE 디코딩 함수: NaN 값은 빈 마스크로 처리하고 정규화 추가
def decode_rle(rle, shape, device):
    if not isinstance(rle, str) or pd.isna(rle):
        # RLE가 문자열이 아니거나 NaN인 경우 빈 마스크 반환
        return torch.zeros(shape, dtype=torch.float32, device=device)
    s = rle.split()
    starts, lengths = [torch.tensor(list(map(int, x)), dtype=torch.int32, device=device) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    mask = torch.zeros(shape[0] * shape[1], dtype=torch.float32, device=device)
    for start, end in zip(starts, ends):
        mask[start:end] = 1
    mask = mask.view(shape[1], shape[0]).T  # Reshape with Fortran order
    # 마스크 정규화
    return (mask - mask.min()) / (mask.max() - mask.min() + 1e-7)

# RLE 인코딩 함수: 바이너리 마스크를 RLE 데이터로 변환
def encode_rle(mask):
    pixels = mask.flatten().cpu().numpy()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

mask_shape = (2048, 2048)  # 마스크의 크기

# CSV 파일 읽어오기
csv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')]
dfs = [pd.read_csv(f) for f in csv_files]

# 앙상블 결과를 저장할 리스트 초기화
final_result = []

# 고유한 image_name과 class 쌍을 처리
unique_keys = dfs[0][['image_name', 'class']].drop_duplicates()

# 모델 가중치 (필요에 따라 조정 가능)
weights = [1.0 for _ in range(len(dfs))]

# 소프트 보팅 수행
for _, row in tqdm(unique_keys.iterrows(), total=len(unique_keys)):
    image_name, cls = row['image_name'], row['class']
    masks = []

    # 각 모델에서 RLE 디코딩 및 가중치 적용
    for idx, df in enumerate(dfs):
        rle = df[(df['image_name'] == image_name) & (df['class'] == cls)]['rle']
        if len(rle) > 0:
            decoded = decode_rle(rle.iloc[0], mask_shape, device)
            masks.append(weights[idx] * decoded)

    # 유효한 마스크가 없으면 처리 건너뛰기
    if not masks:
        continue

    # 가중 평균 계산
    masks_stack = torch.stack(masks)
    average_mask = masks_stack.sum(dim=0) / sum(weights)

    # 평균 마스크를 이진 마스크로 변환 (임계값 적용)
    final_mask = (average_mask > 0.25).float()  # 임계값 0.3으로 설정

    # 최종 마스크를 RLE로 변환
    final_rle = encode_rle(final_mask)

    # 결과 저장
    final_result.append({'image_name': image_name, 'class': cls, 'rle': final_rle})

100%|██████████| 8352/8352 [09:14<00:00, 15.05it/s]


In [5]:
# 결과를 DataFrame으로 변환하여 CSV로 저장
final_result_df = pd.DataFrame(final_result)
final_result_df.to_csv(output_csv, index=False)

print(f"Soft voting ensemble completed. Results saved to {output_csv}")

Soft voting ensemble completed. Results saved to csv_ensemble_soft_1.csv
